## Import

In [1]:
import random
import wandb
wandb.login()
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="Dacon-papering-competition",
    
    # track hyperparameters and run metadata
    config={
    'LEARNING_RATE':3e-4,
    'IMG_SIZE':224,
    "architecture": "Efficientnet-b0",
    'EPOCHS':50,
    'SEED':42,
    'BATCH_SIZE':132,
    "dataset": "Dacon-papering-competition",
    }# hyperparameter
)
# hyperparameter
CFG = wandb.config
CFG

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anthoniy. Use `wandb login --relogin` to force relogin


{'LEARNING_RATE': 0.0003, 'IMG_SIZE': 224, 'architecture': 'Efficientnet-b0', 'EPOCHS': 50, 'SEED': 42, 'BATCH_SIZE': 132, 'dataset': 'Dacon-papering-competition'}

In [2]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

# torch.multiprocessing import
from torch import multiprocessing
from collections import OrderedDict, Counter
from sklearn.utils import resample
import warnings
warnings.filterwarnings(action='ignore') 

c:\Users\DoSungjin\anaconda3\envs\gpu-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Device & Path

In [3]:
# Set the device to CPU or GPU depending on availability
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
torch.cuda.is_available()

True

In [4]:
# 경로지정
import os
os.chdir('../DATA')
os.getcwd()

'c:\\Users\\DoSungjin\\Documents\\GitHub\\Dacon_papering_classification\\DATA'

## Fixed RandomSeed

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [6]:
import os

# 데이터셋 디렉토리 경로
dataset_dir = "new_train"

# 모든 이미지 파일 경로 리스트
all_img_list = []
folder_list = []
train_file_list = os.listdir(dataset_dir)
for item in train_file_list:
    item_path = os.path.join(dataset_dir, item)
    for file in os.listdir(item_path):
        all_img_list.append(os.path.join(item_path, file))
        folder_list.append(item)

# # folder name list
ori_names = os.listdir('ori_train')

new_names = ['furniture_repair', 'cleaning_mop_holder_repair', 'mold', 'twist', 'rust_contamination', 'wobbling', 'fabric_defect', 'molding_repair', 'stain', 'plaster_repair', 'pollution', 'typographical_error', 'crying', 'defective_joint', 'windowdoor_frame_repair', 'cracking', 'excessive_gap', 'piece', 'damage']

# print(len(ori_names), len(new_names))
# os.chdir('new_train')
# # # folder rename
# for ori_name, new_name in zip(ori_names, new_names):
#     os.rename(ori_name, new_name)

# print(os.listdir())
# os.chdir('../')
for a,b in zip(ori_names, new_names):
    len_a = len(os.listdir(f'ori_train/{a}'))
    len_b = len(os.listdir(f'new_train/{b}'))
    if len_a == len_b:
        print('go on')
    else: 
        print('stop')
        
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('\\')[1])

group_100_or_less = []
group_100_to_600 = []
group_600_or_more = []

for key, value in Counter(df['label']).items():
    if value <= 100:
        group_100_or_less.append(key)
    elif value <= 600:
        group_100_to_600.append(key)
    else:
        group_600_or_more.append(key)

# 결과 출력
print("100개 이하 그룹:", group_100_or_less)
print("100~600개 그룹:", group_100_to_600)
print("600개 이상 그룹:", group_600_or_more)
from sklearn.utils import resample

# 100개 이하 그룹 오버샘플링
minor_class_samples_100_or_less = df[df['label'].isin(group_100_or_less)]
minor_class_count_100_or_less = len(minor_class_samples_100_or_less)
oversample_count_100_or_less = int(minor_class_count_100_or_less * 5)
oversampled_minor_class_100_or_less = resample(minor_class_samples_100_or_less, replace=True, n_samples=oversample_count_100_or_less, random_state=42)

# 100~600개 그룹 오버샘플링
minor_class_samples_100_to_600 = df[df['label'].isin(group_100_to_600)]
minor_class_count_100_to_600 = len(minor_class_samples_100_to_600)
oversample_count_100_to_600 = int(minor_class_count_100_to_600 * 1.5)
oversampled_minor_class_100_to_600 = resample(minor_class_samples_100_to_600, replace=True, n_samples=oversample_count_100_to_600, random_state=42)

# 600개 이상 그룹 오버샘플링
minor_class_samples_600_or_more = df[df['label'].isin(group_600_or_more)]
oversampled_minor_class_600_or_more = minor_class_samples_600_or_more  # 그대로 유지

# 오버샘플링된 소수 클래스 데이터셋과 다수 클래스 데이터셋 합치기
df_oversampled = pd.concat([oversampled_minor_class_100_or_less, oversampled_minor_class_100_to_600, oversampled_minor_class_600_or_more])

# 샘플 순서 랜덤하게 섞기
df_oversampled = df_oversampled.sample(frac=1, random_state=42).reset_index(drop=True)

from sklearn.model_selection import train_test_split

train, val, _, _ = train_test_split(df_oversampled, df_oversampled['label'], test_size=0.2, stratify=df_oversampled['label'], random_state=CFG['SEED'])



go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
100개 이하 그룹: ['crying', 'defective_joint', 'excessive_gap', 'fabric_defect', 'furniture_repair', 'piece', 'plaster_repair', 'rust_contamination', 'stain', 'windowdoor_frame_repair', 'wobbling']
100~600개 그룹: ['cleaning_mop_holder_repair', 'cracking', 'mold', 'molding_repair', 'pollution', 'twist', 'typographical_error']
600개 이상 그룹: ['damage']


In [7]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])
train

,img_path,label
3584,new_train\pollution\388.png,12
4431,new_train\damage\842.png,3
2227,new_train\mold\9.png,8
1164,new_train\crying\2.png,2
1171,new_train\fabric_defect\72.png,6
...,...,...
2720,new_train\piece\33.png,10
5319,new_train\damage\520.png,3
5280,new_train\typographical_error\99.png,16
3444,new_train\wobbling\17.png,18


In [8]:
Counter(train['label'])

Counter({12: 778,
         3: 1124,
         8: 154,
         2: 95,
         6: 371,
         0: 348,
         17: 113,
         16: 162,
         9: 157,
         11: 231,
         15: 255,
         18: 214,
         10: 201,
         1: 174,
         4: 71,
         7: 63,
         5: 20,
         13: 55,
         14: 10})

## CustomDataset

In [9]:
from torch.autograd import Variable

class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int)): self.alpha = torch.Tensor([alpha]*19)
        self.alpha[18] = 1-alpha
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * at

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

In [10]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [11]:

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2


train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.HorizontalFlip(p=0.5),
                            A.RandomBrightnessContrast(p=0.5),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [12]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Model Define

In [13]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=18):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)
        self.backbone.fc = nn.Identity()  # Remove the original classifier
        self.classifier = nn.Linear(1280, num_classes)  # Adjust the input size
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x



In [14]:
next(iter(train_loader))[1][0]

tensor(12, dtype=torch.int32)

In [15]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int)): self.alpha = torch.Tensor([alpha]*19)
        self.alpha[18] = 1-alpha
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * at

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

## Train

In [16]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = FocalLoss(gamma=2, alpha=0.25)
    best_score = 0
    best_model = None
    
    # Initialize wandb run
    wandb.init(project='your-project-name', name='training-run')
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
        
        # Log metrics to wandb
        wandb.log({'epoch': epoch, 'train_loss': _train_loss, 'val_loss': _val_loss, 'val_score': _val_score})
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    # Save best model and upload it to wandb
    wandb.save('best_model.pth')
    wandb.finish()
    
    return best_model

In [17]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            label_map = {label: i for i, label in enumerate(set(labels.tolist()))}
            numerical_labels = [label_map[int(label.item())] for label in labels]
            labels = torch.tensor(numerical_labels, dtype=torch.long)
            labels = labels.to(device)


            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

In [18]:
next(iter(train_loader))[0].shape

torch.Size([132, 3, 224, 224])

## Run!!

In [19]:
model = BaseModel()
model.eval()
optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/35 [00:08<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 8.00 GiB total capacity; 7.13 GiB already allocated; 0 bytes free; 7.29 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [25]:
!nvidia-smi


Fri May 12 16:54:03 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.61                 Driver Version: 531.61       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070 L...  WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8               10W /  N/A|   8022MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [21]:
torch.cuda.empty_cache()

In [26]:
import torch
import gc

# If you are using CUDA
torch.cuda.empty_cache()
gc.collect()

0

## Inference

In [ ]:
test = pd.read_csv('test.csv')
test

In [ ]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [ ]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/25 [00:00<?, ?it/s]

## Submission

In [ ]:
submit = pd.read_csv('sample_submission.csv')

In [ ]:
submit['label'] = preds

In [ ]:
# result folder 생성
from datetime import datetime, timezone, timedelta

kst = timezone(timedelta(hours=9))
train_serial =  datetime.now(kst).strftime('%Y%m%d_%H%M%S')

Record_path = os.path.join('result', train_serial)

os.makedirs(Record_path, exist_ok=True)




In [ ]:
# Dictionary to map old column names to new column names
name_map = dict(zip(new_names, ori_names))
name_map


# Rename columns using the rename() method
submit_test = submit.replace(name_map)
submit_test

In [ ]:
submit_test.to_csv(os.path.join(Record_path,'submission.csv'), index=False)

In [ ]:
from collections import Counter
Counter(submit_test['label'])

In [ ]:
# model information 저장
import json
model_info = dict(CFG)
with open(os.path.join(Record_path,'model_info.json'), 'w') as f:
    json.dump(model_info, f)